In [1]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train import *
from evaluation import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



/Users/nikolaushouben/opt/anaconda3/envs/Wattcast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: /Users/nikolaushouben/Desktop/WattCast


In [2]:
def train_eval_tuning():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(init_config)
    config = wandb.config

    print("Getting data...")

    data = load_data(config)
    config = derive_config_params(config)
    piped_data, pipeline = data_pipeline(config, data)


    _, _, ts_test_piped, _, _, ts_test_weather_piped = piped_data
    longest_ts_test_idx = get_longest_subseries_idx(ts_test_piped)
    trg_test_inversed = pipeline.inverse_transform(ts_test_piped)[longest_ts_test_idx]

    print("Getting model instance...")
    model_instance = get_model(config)
    model_instance, _ = train_models(config, {config.model : model_instance}, {config.model: config}) # need to pass in a dict of models and configs

    print("Evaluating model...")
    predictions, score = predict_testset(model_instance[0], 
                                  ts_test_piped[longest_ts_test_idx], 
                                  ts_test_weather_piped[longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()


### Running

In [ ]:
locations, _ = get_hdf_keys(dir_path)
list(((list(locations.keys())[0].split(".")[0], location) for location in locations['2_town.h5']))

In [ ]:
#we keep the number of sweeps constant to ensure a fair playing field for all algorithms
sweeps = 3 # number of sweeps per model, per location: 

scale_location_pairs = [
('2_town', 'GLENDOVEER-13596'),
# ('2_town.h5', 'GLENDOVEER-13597'),
 #('2_town.h5', 'GLENDOVEER-13598'),
 #('2_town.h5', 'GLENDOVEER-13599'),
 #('2_town.h5', 'GLENDOVEER-CLIFFGATE'),
 #('2_town.h5', 'GLENDOVEER-NORTHEAST'),
 #('2_town', 'KELLY'),
 #('2_town', 'LENTS-13101'),
 #('2_town.h5', 'LENTS-HAPPY'),
 #('2_town.h5', 'LENTS-MT'),
 #('2_town.h5', 'LENTS-NORTH'),
 #('2_town', 'MIDWAY-DIVISION'),
 #('2_town.h5', 'MIDWAY-DOUGLAS'),
 #('2_town.h5', 'MIDWAY-LYNCH'),
 #('2_town.h5', 'MIDWAY-POWELLHURST'),
 #('2_town', 'RAMAPO-EMERALD'),
 #('2_town.h5', 'RAMAPO-GILBERT'),
 #('2_town.h5', 'RAMAPO-RAMAPO')
 ]

models = [
        #'rf',
        'xgb', 
        #'gru', 
        #'lgbm',  
        #'nbeats',
        #'tft'
        ]


for scale, location in scale_location_pairs:

    for model in models:
        # placeholder initialization of config file (will be updated in train_eval_light())
        init_config = {
                        'spatial_scale': "2_town",
                        'temp_resolution': 60,
                        'location': 'GLENDOVEER-13596',
                        'model': model,
                        'horizon_in_hours': 24,
                        'lookback_in_hours': 24,
                        'boxcox': True,
                        'liklihood': None,
                        'weather_available': True,
                        'datetime_encodings': True,
                        'heat_wave_binary': True,
                        'datetime_attributes': ["dayofweek", "week"],
                        'use_cov_as_past_cov': False,
                    }
        
        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + init_config['spatial_scale'] + '_' + init_config['location'] + '_' + str(init_config['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="Portland_AMI_tuning")
        wandb.agent(sweep_id, train_eval_tuning, count=sweeps)
        wandb.finish()



Create sweep with ID: vki08zrm
Sweep URL: https://wandb.ai/wattcast/Portland_AMI_tuning/sweeps/vki08zrm


wandb: Agent Starting Run: 8icu8a5y with config:
wandb: 	datetime_encodings: 0
wandb: 	heat_wave_binary: 1
wandb: 	learning_rate: 0.1
wandb: 	lookback_in_hours: 24
wandb: 	max_depth: 2
wandb: 	min_child_weight: 5
wandb: 	n_estimators: 500
wandb: 	objective: reg:pseudohubererror
wandb: 	reg_lambda: 2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'lookback_in_hours' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'datetime_encodings' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'heat_wave_binary' was locked by 'sweep' (ignored update).


Getting data...
Getting model instance...
Initializing kwargs for sklearn-API type model xgb...
['min_child_weight', 'n_estimators', 'verbosity', 'objective', 'early_stopping_rounds', 'eval_metric', 'reg_lambda', 'learning_rate', 'max_depth']
Training XGBModel
[0]	validation_0-rmse:0.22493
[1]	validation_0-rmse:0.20644
[2]	validation_0-rmse:0.19003
[3]	validation_0-rmse:0.17525
[4]	validation_0-rmse:0.16226
[5]	validation_0-rmse:0.15115
[6]	validation_0-rmse:0.14128
[7]	validation_0-rmse:0.13234
[8]	validation_0-rmse:0.12463
[9]	validation_0-rmse:0.11712
[10]	validation_0-rmse:0.11052
[11]	validation_0-rmse:0.10438
[12]	validation_0-rmse:0.09926
[13]	validation_0-rmse:0.09455
[14]	validation_0-rmse:0.08982
[15]	validation_0-rmse:0.08636
[16]	validation_0-rmse:0.08340
[17]	validation_0-rmse:0.07985
[18]	validation_0-rmse:0.07649
[19]	validation_0-rmse:0.07432
[20]	validation_0-rmse:0.07200
[21]	validation_0-rmse:0.06911
[22]	validation_0-rmse:0.06706
[23]	validation_0-rmse:0.06584
[24]	

eval_loss,▁
eval_loss,0.07245


...

Getting data...
Getting model instance...
Initializing kwargs for sklearn-API type model xgb...
['min_child_weight', 'n_estimators', 'verbosity', 'objective', 'early_stopping_rounds', 'eval_metric', 'reg_lambda', 'learning_rate', 'max_depth']
Training XGBModel
[0]	validation_0-rmse:0.19494
[1]	validation_0-rmse:0.15672
[2]	validation_0-rmse:0.12756
[3]	validation_0-rmse:0.10526
[4]	validation_0-rmse:0.08730
[5]	validation_0-rmse:0.07356
[6]	validation_0-rmse:0.06335
[7]	validation_0-rmse:0.05518
[8]	validation_0-rmse:0.04895
[9]	validation_0-rmse:0.04441
[10]	validation_0-rmse:0.04099
[11]	validation_0-rmse:0.03831
[12]	validation_0-rmse:0.03635
[13]	validation_0-rmse:0.03493
[14]	validation_0-rmse:0.03388
[15]	validation_0-rmse:0.03314
[16]	validation_0-rmse:0.03245
[17]	validation_0-rmse:0.03206
[18]	validation_0-rmse:0.03183
[19]	validation_0-rmse:0.03150
[20]	validation_0-rmse:0.03118
[21]	validation_0-rmse:0.03100
[22]	validation_0-rmse:0.03088
[23]	validation_0-rmse:0.03081
[24]	

wandb: Ctrl + C detected. Stopping sweep.


[20]	validation_0-rmse:0.04667
[21]	validation_0-rmse:0.04632
[22]	validation_0-rmse:0.04640
[23]	validation_0-rmse:0.04610
[24]	validation_0-rmse:0.04591
[25]	validation_0-rmse:0.04576
[26]	validation_0-rmse:0.04549
[27]	validation_0-rmse:0.04545
[28]	validation_0-rmse:0.04539
[29]	validation_0-rmse:0.04537
[30]	validation_0-rmse:0.04528
[31]	validation_0-rmse:0.04517
[32]	validation_0-rmse:0.04492
[33]	validation_0-rmse:0.04490
[34]	validation_0-rmse:0.04475
[35]	validation_0-rmse:0.04477
[36]	validation_0-rmse:0.04476
[0]	validation_0-rmse:0.20209
[1]	validation_0-rmse:0.16906
[2]	validation_0-rmse:0.14506
[3]	validation_0-rmse:0.12625
[4]	validation_0-rmse:0.11125
[5]	validation_0-rmse:0.10066
[6]	validation_0-rmse:0.09088
[7]	validation_0-rmse:0.08351
[8]	validation_0-rmse:0.07847
[9]	validation_0-rmse:0.07420
[10]	validation_0-rmse:0.07100
[11]	validation_0-rmse:0.06849
[12]	validation_0-rmse:0.06560
[13]	validation_0-rmse:0.06415
[14]	validation_0-rmse:0.06261
[15]	validation_0-

[29]	validation_0-rmse:0.08078
[30]	validation_0-rmse:0.08072
[31]	validation_0-rmse:0.08065
[32]	validation_0-rmse:0.08079
[33]	validation_0-rmse:0.08058
[34]	validation_0-rmse:0.08064
[35]	validation_0-rmse:0.08045


[36]	validation_0-rmse:0.08033
[37]	validation_0-rmse:0.08019
[38]	validation_0-rmse:0.08021
[0]	validation_0-rmse:0.20553
[1]	validation_0-rmse:0.17457
[2]	validation_0-rmse:0.15285
[3]	validation_0-rmse:0.13677
[4]	validation_0-rmse:0.12567
[5]	validation_0-rmse:0.11628
[6]	validation_0-rmse:0.11019
[7]	validation_0-rmse:0.10541
[8]	validation_0-rmse:0.10194
[9]	validation_0-rmse:0.09923
[10]	validation_0-rmse:0.09744
[11]	validation_0-rmse:0.09561
[12]	validation_0-rmse:0.09425
[13]	validation_0-rmse:0.09319
[14]	validation_0-rmse:0.09205
[15]	validation_0-rmse:0.09109
[16]	validation_0-rmse:0.09028
[17]	validation_0-rmse:0.08971
[18]	validation_0-rmse:0.08924
[19]	validation_0-rmse:0.08899
[20]	validation_0-rmse:0.08895
[21]	validation_0-rmse:0.08870
[22]	validation_0-rmse:0.08840
[23]	validation_0-rmse:0.08800
[24]	validation_0-rmse:0.08764
[25]	validation_0-rmse:0.08744
[26]	validation_0-rmse:0.08709
[27]	validation_0-rmse:0.08674
[28]	validation_0-rmse:0.08640
[29]	validation_0-

### Degbugging